# Studio: Working with Databases in Python

For today's studio, we will be using the [TV Shows dataset](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney) from Kaggle. We have already downloaded the CSV for you.

You will also be using the watchlist you just created to figure out which streaming services contain the shows that you want to watch next so you can decide which one is worth the money to you.

As you complete the different tasks in the studio, you may choose between using Pandas or SQL. Remember that during the prep work, we learned that one is oftentimes more efficient at certain tasks than the other, so choose wisely!

## My Watchlist

If you would like, please use this space to make note of your watchlist by editing the text cell.

Ozark
Schitts Creek
The Office
New Girl
This is Us
Happy Endings
Younger
Good Trouble
The Bold Type
Dope Sick

## Database Setup

To start with, import the necessary libraries and create a dataframe from the provided CSV. Print the info out for the dataframe. After that, you may drop the column called `Unnamed: 0` and rename any columns with spaces in the names or unusual characters such as `"+"`. Print out the info for the dataframe again to ensure that your changes were effective.

In [2]:
import pandas as pd
import sqlite3 as sl

data= pd.read_csv("tv_shows.csv")
data= data.drop('Unnamed: 0', 1)
data=data.rename(columns={'Rotten Tomatoes':'Rotten', 'Disney+': 'Disney', 'Prime Video':'Prime'})
data.head()

/var/folders/5v/128x261d72sb4dsy21d7f77c0000gn/T/ipykernel_7125/238617221.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data= data.drop('Unnamed: 0', 1)


,ID,Title,Year,Age,IMDb,Rotten,Netflix,Hulu,Prime,Disney,Type
0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


With your dataframe at the ready, create a new database called `tv`. Add a new table to your database called `shows` using the data in the dataframe. 

In [3]:
con=sl.connect('tv.db')

In [4]:
data.to_sql('show1', con) 

ValueError: Table 'show1' already exists.

With your new table and database set up, print out the top 20 records in the `shows` table.

In [5]:
print(data.head(20))

with con:
        data1 = con.execute("SELECT * FROM show1 WHERE id<=20")
        for row in data1:
            print(row)

    ID                       Title  Year  Age    IMDb   Rotten  Netflix  Hulu  \
0    1                Breaking Bad  2008  18+  9.4/10  100/100        1     0   
1    2             Stranger Things  2016  16+  8.7/10   96/100        1     0   
2    3             Attack on Titan  2013  18+  9.0/10   95/100        1     1   
3    4            Better Call Saul  2015  18+  8.8/10   94/100        1     0   
4    5                        Dark  2017  16+  8.8/10   93/100        1     0   
5    6  Avatar: The Last Airbender  2005   7+  9.3/10   93/100        1     0   
6    7              Peaky Blinders  2013  18+  8.8/10   93/100        1     0   
7    8            The Walking Dead  2010  18+  8.2/10   93/100        1     0   
8    9                Black Mirror  2011  18+  8.8/10   92/100        1     0   
9   10          The Queen's Gambit  2020  18+  8.6/10   92/100        1     0   
10  11                  Mindhunter  2017  18+  8.6/10   90/100        1     0   
11  12                   Com

Now add a table that includes an id number and the titles on your watchlist.

In [6]:
with con:
    con.execute('''
        CREATE TABLE watchlist2(
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            title TEXT
        );
    ''')
    
    sql= 'INSERT INTO watchlist2 (id, title) values (?, ?)'
    data2=[
        (1, 'Ozark'),
        (2, 'Schitts Creek'),
        (3, 'The Office'),
        (4, 'New Girl'),
        (5, 'This is Us'),
        (6, 'Happy Endings'),
        (7, 'Younger'),
        (8, 'Good Trouble'),
        (9, 'The Bold Type'),
        (10, 'Dope Sick')
    ]
    
    con.executemany(sql, data2)
    data3=(con.execute('SELECT * FROM watchlist2'))
    for row in data3:
        print(row)
    
    

OperationalError: table watchlist2 already exists

## Working with the Data

Using either Pandas or SQL, determine how many shows are on each streaming service and what the mean is for each streaming service. 

**Note**: You may notice that the mean represents the percentage of shows in the dataframe that are on each streaming service. Take a moment to think about why that is.

In [7]:
print(data['Rotten'].value_counts())
print(data['Netflix'].value_counts())
print(data['Hulu'].value_counts())
print(data['Disney'].value_counts())
print(data['Prime'].value_counts())

10/100     304
13/100     174
45/100     135
51/100     131
52/100     125
          ... 
91/100       4
94/100       2
95/100       2
100/100      2
96/100       1
Name: Rotten, Length: 85, dtype: int64
0    3397
1    1971
Name: Netflix, dtype: int64
0    3747
1    1621
Name: Hulu, dtype: int64
0    5017
1     351
Name: Disney, dtype: int64
0    3537
1    1831
Name: Prime, dtype: int64


Join your watchlist data with the shows data to determine which streaming services your watchlist shows are on and make a new table in your database using the joined data. Print out the data in your joined table to see what shows on your watchlist on in the original dataset. With the joined data, determine the percentage of your watchlist shows that are on each streaming service and how many of your watchlist shows are on each streaming service.

In [15]:
with con:
    join_data=con.execute('''SELECT * 
                             FROM show1
                             LEFT JOIN watchlist2
                             ON show1.title=watchlist2.title
                        ;''')
    
    for row in join_data:
        print(row)


(0, 1, 'Breaking Bad', 2008, '18+', '9.4/10', '100/100', 1, 0, 0, 0, 1, None, None)
(1, 2, 'Stranger Things', 2016, '16+', '8.7/10', '96/100', 1, 0, 0, 0, 1, None, None)
(2, 3, 'Attack on Titan', 2013, '18+', '9.0/10', '95/100', 1, 1, 0, 0, 1, None, None)
(3, 4, 'Better Call Saul', 2015, '18+', '8.8/10', '94/100', 1, 0, 0, 0, 1, None, None)
(4, 5, 'Dark', 2017, '16+', '8.8/10', '93/100', 1, 0, 0, 0, 1, None, None)
(5, 6, 'Avatar: The Last Airbender', 2005, '7+', '9.3/10', '93/100', 1, 0, 1, 0, 1, None, None)
(6, 7, 'Peaky Blinders', 2013, '18+', '8.8/10', '93/100', 1, 0, 0, 0, 1, None, None)
(7, 8, 'The Walking Dead', 2010, '18+', '8.2/10', '93/100', 1, 0, 0, 0, 1, None, None)
(8, 9, 'Black Mirror', 2011, '18+', '8.8/10', '92/100', 1, 0, 0, 0, 1, None, None)
(9, 10, "The Queen's Gambit", 2020, '18+', '8.6/10', '92/100', 1, 0, 0, 0, 1, None, None)
(10, 11, 'Mindhunter', 2017, '18+', '8.6/10', '90/100', 1, 0, 0, 0, 1, None, None)
(11, 12, 'Community', 2009, '7+', '8.5/10', '90/100', 1, 1

(1358, 1362, 'The Fix', 2018, '18+', '7.1/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1359, 1363, 'One More Time', 2016, None, '6.9/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1360, 1364, 'Astronomy Club: The Sketch Show', 2019, '18+', '7.1/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1361, 1365, 'Prince of Peoria', 2018, 'all', '5.9/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1362, 1366, 'Miss in Kiss', 2016, None, '7.6/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1363, 1367, 'Numberblocks', 2017, 'all', '8.1/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1364, 1368, 'Polly Pocket', 2018, '7+', '6.8/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1365, 1369, 'All About the Washingtons', 2018, 'all', '4.5/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1366, 1370, 'Goedam', 2020, None, '5.8/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1367, 1371, "Archibald's Next Big Thing", 2019, '7+', '7.8/10', '45/100', 1, 0, 0, 0, 1, None, None)
(1368, 1372, 'The Platform', 2020, None, '7.0/10', '45/100', 1, 0, 0, 0, 1,

(2328, 2393, 'Highlander: The Series', 1992, '16+', '7.2/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2329, 2395, 'Prison School', 2015, '18+', '7.6/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2330, 2396, 'Good Luck Charlie', 2010, 'all', '7.0/10', '65/100', 0, 1, 0, 1, 1, None, None)
(2331, 2397, 'Medium', 2005, '16+', '7.0/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2332, 2398, 'Fire Force', 2019, '16+', '7.7/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2333, 2399, 'Second Chance', 2016, '16+', '7.5/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2334, 2400, 'American Housewife', 2016, '7+', '7.4/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2335, 2401, 'The First', 2018, '18+', '6.7/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2336, 2403, 'Reprisal', 2019, '18+', '7.5/10', '65/100', 0, 1, 0, 0, 1, None, None)
(2337, 2404, 'Mushoku Tensei: Jobless Reincarnation', 2021, '16+', '8.3/10', '64/100', 0, 1, 0, 0, 1, None, None)
(2338, 2405, 'The Kids Are Alright', 2018, '7+', '7.9/10', '64/100', 0, 1, 0, 

(3357, 3490, 'GMA Day', 2018, None, None, '16/100', 0, 1, 0, 0, 1, None, None)
(3358, 3491, 'Magnificent Obsessions', 2014, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3359, 3492, 'Jeremiah Bullfrog Forks It', 2017, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3360, 3493, "MOJO's The Circuit", 2008, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3361, 3494, 'ABC News Documentaries', 2015, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3362, 3495, "Asia's Underworld", 2012, None, None, '14/100', 0, 1, 1, 0, 1, None, None)
(3363, 3496, 'pocket.watch CaptainSparklez mishmash', 2018, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3364, 3497, 'Trials of the Wild', 2018, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3365, 3498, 'Ventaneando', 2012, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3366, 3499, 'Truth and Lies: Watergate', 2017, None, None, '14/100', 0, 1, 0, 0, 1, None, None)
(3367, 3500, 'Bondi Harvest', 2018, None, None, '14/100', 0, 1, 0, 0, 1, None

(4165, 4460, "Seven Senses of the Re'Union", 2018, None, '4.6/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4166, 4461, 'The Timeline', 2015, None, '8.2/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4167, 4462, 'Roughing It', 2002, None, '7.0/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4168, 4463, 'Sneg i pepel', 2015, None, '6.2/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4169, 4464, 'One Mic Stand', 2019, None, '6.6/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4170, 4465, 'Bat Pat', 2016, None, None, '38/100', 0, 0, 1, 0, 1, None, None)
(4171, 4466, 'Secrets of Archaeology', 2014, None, '8.0/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4172, 4467, 'Greatest Tank Battles', 2010, '7+', '7.4/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4173, 4468, 'Nightmare in Suburbia', 2008, None, '7.9/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4174, 4469, 'Extinct', 2001, None, '7.3/10', '38/100', 0, 0, 1, 0, 1, None, None)
(4175, 4470, "We'll Meet Again", 1982, '7+', '7.1/10', '38/100', 0, 0, 1, 0, 1, None, None

(4900, 5219, 'The History Of Ford Trucks', 2016, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4901, 5220, 'Car Collectors', 2013, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4902, 5221, 'Bana: Heart of Darkness', 2017, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4903, 5222, "Grandma's Cats (Are Trying To Kill Her!)", 2015, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4904, 5223, 'Minecraft - Survival Madness Adventures', 2016, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4905, 5224, 'Empires Of Stone', 2010, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4906, 5225, 'Beyond Invention', 2004, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4907, 5226, 'Railway Roundup', 2010, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4908, 5227, 'Review: Disney Toy Review', 2017, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4909, 5228, 'Delinquent Hamsters', 2013, None, None, '10/100', 0, 0, 1, 0, 1, None, None)
(4910, 5229, 'Review: Cool Collectibles Dolls, 

## Results

Now that you have done your analysis, make note of the answers to the following questions by editing the text cell:

1. Was every show on your watchlist in the Kaggle dataset? Do you have any ideas as to why a show might not have been present?
2. Did you include a show or shows in your watchlist that is exclusive to one of the platforms? How might that have impacted your analysis?
2. Which streaming service(s) offered the most shows on your watchlist? Which streaming service(s) offered the least?
3. Based on the shows you want to watch and the results of your analysis, is there a streaming service you think would be a good fit for you?